In [1]:
import mediapipe as mp 
import cv2

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [53]:
camera = cv2.VideoCapture(1)
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands = 1) as hands:
    while camera.isOpened():
        _, image = camera.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 
        results = hands.process(image)

        image.flags.writeable = True
        landmark_list = []
        if results.multi_hand_landmarks:
            for landmark in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, landmark, mp_hands.HAND_CONNECTIONS)
            
        cv2.imshow("Camera No. 1", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        if cv2.waitKey(10) & 0xff == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()

In [30]:
num_coord = len(results.multi_hand_landmarks[0].landmark)
num_coord

21

In [82]:
import csv
import numpy as np 
import os

In [73]:
landmarks = ['class']
for num in range(1, num_coord + 1):
    landmarks += [f'x{num}', f'y{num}', f'z{num}'] 

In [48]:
landmarks
len(landmarks)

85

In [74]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)
    

In [90]:
class_id = "rock on"

In [91]:
camera = cv2.VideoCapture(1)
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands = 1) as hands:
    while camera.isOpened():
        _, image = camera.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 
        results = hands.process(image)

        image.flags.writeable = True
        landmark_list = []
        if results.multi_hand_landmarks:
            for landmark in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, landmark, mp_hands.HAND_CONNECTIONS)
        
        try:
            hand = enumerate(results.multi_hand_landmarks[0].landmark)
            hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for _, landmark in hand]).flatten())
            hand_row.insert(0, class_id)
            #print(hand_row)
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(hand_row)
                
        except Exception as e:
            pass
            
        cv2.imshow("Camera No. 1", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        if cv2.waitKey(10) & 0xff == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()

In [92]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [93]:
df = pd.read_csv('coords.csv')

In [94]:
df.head()

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z18,x19,y19,z19,x20,y20,z20,x21,y21,z21
0,thumbs up,0.801459,0.822324,-0.000058,0.726228,0.770119,0.048023,0.689402,0.672562,0.088087,...,0.071743,0.827936,0.566918,0.130440,0.820741,0.606786,0.141728,0.819270,0.640020,0.135834
1,thumbs up,0.764270,0.811680,-0.000050,0.724259,0.648953,0.014841,0.701628,0.505037,-0.015446,...,-0.158971,0.843282,0.654111,-0.131476,0.840623,0.678846,-0.098110,0.828789,0.711179,-0.089521
2,thumbs up,0.794234,0.778502,0.000017,0.755896,0.618299,0.050981,0.724439,0.487586,0.062548,...,-0.017929,0.779653,0.573631,0.049431,0.780284,0.598163,0.078384,0.786201,0.633775,0.082501
3,thumbs up,0.781435,0.758817,-0.000016,0.741555,0.607730,0.020929,0.713720,0.461251,-0.005593,...,-0.219740,0.756638,0.628110,-0.229306,0.755878,0.652784,-0.209067,0.762889,0.676525,-0.200228
4,thumbs up,0.776986,0.737279,-0.000128,0.720864,0.601111,0.005459,0.701665,0.448895,-0.025487,...,-0.219658,0.723151,0.629071,-0.228606,0.723751,0.648864,-0.202384,0.740403,0.642477,-0.190361


In [95]:
df.tail()

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z18,x19,y19,z19,x20,y20,z20,x21,y21,z21
1856,rock on,0.298758,0.654636,-0.000086,0.341620,0.618174,-0.035347,0.370802,0.552308,-0.058708,...,-0.023314,0.277427,0.354912,-0.035740,0.270054,0.302638,-0.045474,0.262006,0.254688,-0.053252
1857,rock on,0.300560,0.656274,-0.000077,0.343342,0.620054,-0.039849,0.372885,0.554434,-0.065052,...,-0.026932,0.279213,0.362475,-0.043028,0.272089,0.309391,-0.054985,0.263621,0.257232,-0.063811
1858,rock on,0.300726,0.660358,-0.000093,0.344563,0.624469,-0.036495,0.373250,0.552493,-0.057714,...,-0.023864,0.279091,0.367754,-0.039097,0.272537,0.315308,-0.049309,0.264446,0.262213,-0.056819
1859,rock on,0.300071,0.657584,-0.000096,0.343346,0.621804,-0.034081,0.373086,0.552123,-0.054568,...,-0.026091,0.278616,0.362044,-0.038113,0.271744,0.309430,-0.046645,0.264250,0.256387,-0.053727
1860,rock on,0.298322,0.655123,-0.000097,0.341954,0.618907,-0.031688,0.371656,0.548993,-0.050835,...,-0.027913,0.277514,0.362200,-0.040783,0.271287,0.308311,-0.049710,0.264470,0.254691,-0.056639


In [96]:
X = df.drop('class', axis=1)
y = df['class']

In [97]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [98]:
x_train

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,...,z18,x19,y19,z19,x20,y20,z20,x21,y21,z21
780,0.454955,0.271477,0.000048,0.462628,0.344303,-0.081103,0.480039,0.411578,-0.141371,0.490973,...,-0.137726,0.533731,0.148255,-0.174442,0.513263,0.172493,-0.183920,0.513399,0.185238,-0.189679
1473,0.545324,0.634085,-0.000080,0.487740,0.616723,-0.017777,0.443834,0.552289,-0.040246,0.435706,...,-0.039026,0.537574,0.295682,-0.051275,0.540472,0.233053,-0.067567,0.546813,0.172102,-0.077186
1570,0.596114,0.623676,-0.000082,0.531638,0.578961,-0.010025,0.480568,0.479790,-0.032385,0.472973,...,-0.059030,0.610353,0.179615,-0.076486,0.617600,0.090857,-0.095003,0.625540,0.010750,-0.111165
1255,0.580628,0.561150,-0.000010,0.628617,0.572799,0.006501,0.670786,0.540321,0.011206,0.698234,...,-0.010143,0.582542,0.270232,-0.018023,0.584063,0.216013,-0.022308,0.586797,0.161229,-0.023976
72,0.486672,0.575971,0.000081,0.454925,0.478828,0.040456,0.411932,0.382906,0.022008,0.383126,...,-0.156308,0.338647,0.610438,-0.131396,0.362466,0.619524,-0.097514,0.381462,0.611851,-0.082269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,0.339606,0.551226,-0.000040,0.399003,0.531939,-0.020390,0.447563,0.484712,-0.038103,0.485938,...,-0.035240,0.287023,0.232654,-0.054104,0.282061,0.176714,-0.070991,0.279556,0.119628,-0.078994
1077,0.872270,0.911171,-0.000058,0.783574,0.817912,0.001000,0.721664,0.737295,-0.018084,0.664793,...,-0.040394,0.888548,0.471408,-0.031006,0.885659,0.399713,-0.016520,0.890967,0.343327,-0.013434
1318,0.254228,0.568834,-0.000006,0.309035,0.546196,-0.030840,0.359323,0.485895,-0.044135,0.401811,...,-0.029066,0.201922,0.250330,-0.053790,0.196241,0.198894,-0.070872,0.193134,0.146809,-0.078953
723,0.731163,0.425013,-0.000017,0.690163,0.452365,-0.011068,0.653201,0.457602,-0.036386,0.628331,...,-0.053635,0.627705,0.266360,-0.041133,0.631760,0.275483,-0.030159,0.641943,0.285923,-0.024528


In [99]:
y_train

780     thumbs down
1473        rock on
1570        rock on
1255           love
72        thumbs up
           ...     
1228           love
1077           love
1318           love
723     thumbs down
815     thumbs down
Name: class, Length: 1302, dtype: object

In [100]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier